In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import copy
import time
import scipy.stats as stats
from scipy.stats import expon

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
jd_offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':jd_offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
jd_collection = []
for cset in clickset_list:
    if cset not in jd_collection:
        jd_collection.append(cset)
print('there are {} different assortments'.format(len(jd_collection)))
print('check offertimes',len(jd_offertimes))

there are 9 different products
there are 134 different assortments
check offertimes 134


In [4]:
def generate_negatively_correlated_covariance_matrix(n):
    # Generate a random correlation matrix with negative correlations
    correlation_matrix = np.random.uniform(low=-0.9, high=-0.1, size=(n, n))
    np.fill_diagonal(correlation_matrix, 1)  # Set diagonal elements to 1
    correlation_matrix = (correlation_matrix + correlation_matrix.T) / 2  # Ensure symmetry

    # Ensure the correlation matrix is valid for Cholesky decomposition
    cholesky_success = False
    while not cholesky_success:
        try:
            # Perform Cholesky decomposition
            cholesky_matrix = np.linalg.cholesky(correlation_matrix)
            cholesky_success = True
        except np.linalg.LinAlgError:
            # If Cholesky decomposition fails, adjust the correlation matrix slightly
            correlation_matrix += np.eye(n) * 0.01  # Adjust diagonal elements slightly

    # Generate random standard deviations
    #standard_deviations = np.random.uniform(0.5, 2, size=n)
    
    variance_outsideopt = np.random.uniform(30,40)
    variance_products = np.random.uniform(0.5,2,size=n-1)
    standard_deviations = np.hstack((variance_outsideopt,variance_products))
    # Construct diagonal matrix from standard deviations
    diagonal_matrix = np.diag(standard_deviations)
    #diagonal_matrix = np.eye(n)

    # Calculate the covariance matrix using Cholesky decomposition and diagonal matrix
    covariance_matrix = cholesky_matrix @ diagonal_matrix @ cholesky_matrix.T

    return covariance_matrix



In [5]:
def is_positive_semidefinite(matrix):
    """
    Check if a matrix is positive semidefinite.
    
    Args:
        matrix (numpy.ndarray): The matrix to check.
        
    Returns:
        bool: True if the matrix is positive semidefinite, False otherwise.
    """
    # Check if the matrix is symmetric
    if not np.allclose(matrix, matrix.T):
        return False
    
    # Check if all eigenvalues are non-negative
    eigvals, _ = np.linalg.eig(matrix)
    return np.all(eigvals >= 0)


In [6]:
def collection_distribution_prob_neg(n,collection):
    # generate mean 
    #mu = np.random.uniform(-2,2,n)
    mu_0 = np.random.uniform(6,10)
    mu_1n = np.random.uniform(-2,2,n-1)
    mu = np.hstack((mu_0,mu_1n))
    # generate covariance matrix with positive correlation
    neg_cov = generate_negatively_correlated_covariance_matrix(n)
    if is_positive_semidefinite(neg_cov): 
        sample_size = 10000
        utility_samples = np.random.multivariate_normal(mu,neg_cov,size=sample_size)
    else:
        print('Negative correlation matrix error')
    
    collection_distribution = np.zeros((n,len(collection)))
    for i in range(len(collection)):
        curr_assortment = collection[i]
        curr_population = [[] for _ in range(sample_size)] 
        
        
        for j in range(sample_size):
            for k in curr_assortment:
                curr_population[j].append(utility_samples[j][k])
                # each sub list records only the utilities of products in the current assortment
                
        frequency = [0]*len(curr_assortment)
        for j in range(sample_size):
            max_index = np.argmax(np.array(curr_population[j]))
            # product is chosen iff the utility of the product is max in the assortment
            frequency[max_index] = frequency[max_index] +1 
            # update the frequency of product to be chosen
            
        prob = np.array(frequency)/np.sum(frequency)
        for j in range(len(curr_assortment)):
            collection_distribution[curr_assortment[j]][i] = prob[j]
            
    return collection_distribution

In [7]:
def whole_instance_generation(n,whole_collection,whole_offertimes):
    ## generate LCmem instances based on the assortment collection infomation
    # step 1: randomly generate LCmem parameters
    # 1.1 the number of classes in LCmem, the number of classes is between 10 and 15, both endpoints are included
    ''' num_classes = 20
    # 1.2 randomly generate weights of all classes 
    #weight_pre = np.random.exponential(1, num_classes)
    weights = np.array([1/ num_classes for _ in range(num_classes)])
    # 1.3 randomly generate parameters of each mem
    parameters_v = np.random.uniform(-30, 30, size=(num_classes,n)) '''
    ''' parameters_v = np.zeros((num_classes,n))
    for i in range(num_classes):
        parameters_v[i] = np.random.exponential(1, n) '''
    #np.random.uniform(-30, 30, size=(num_classes,n))
    # step 2: generate LCmem instance with the above parameters of LCmem
    true_instance = collection_distribution_prob_neg(n,whole_collection)
    # step 3: generate multinomial samples based on lcmem instance and the emprical assortment offertimes
    purchased_samples = []
    for i in range(len(whole_collection)):
        sample_i = np.random.multinomial(whole_offertimes[i], true_instance[:,i])
        purchased_samples.append(sample_i)
    # step 4: compute the simulated collection probabilities
    whole_choice_collection = np.zeros((n,len(whole_collection)))
    for i in range(len(whole_collection)):
        whole_choice_collection[:,i] = np.array([k/np.sum(purchased_samples[i]) for k in purchased_samples[i]])
        
    return whole_choice_collection,purchased_samples

In [8]:
def filter_collection_offertimes(whole_collection,whole_offertimes,least_offetimes):
    
    collection = []
    offertimes = []
    assortment_index = []
    
    for i in range(len(whole_offertimes)):
        if whole_offertimes[i]>=least_offetimes:
            collection.append(whole_collection[i])
            offertimes.append(whole_offertimes[i])
            assortment_index.append(i)
            
    return collection,offertimes,assortment_index

In [9]:
pred_offer_times_list = [20,30,40,50,60]
pred_test_collection_size = [5,4,3,3,2]
pred_train_collection_size = [24,20,16,12,11]
pred_instance_size = [50,50,50,50,50]
price = np.array([0,1.041,0.456,0.391,1.657,1.174,0.474,0.67,1.522])

In [10]:

all_full_collections = []
all_full_offertimes = []
all_full_assortment_index = []
full_collection_size = []
for i in range(len(pred_offer_times_list)):
    collection, offertimes, assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[i])
    all_full_collections.append(collection)
    all_full_offertimes.append(offertimes)
    all_full_assortment_index.append(assortment_index)
    full_collection_size.append(len(assortment_index))
    print("number of assortments with offertimes {} is {}".format(pred_offer_times_list[i], len(assortment_index)))

number of assortments with offertimes 20 is 29
number of assortments with offertimes 30 is 24
number of assortments with offertimes 40 is 19
number of assortments with offertimes 50 is 15
number of assortments with offertimes 60 is 13


In [11]:
# check if the collection are nested
def check_subsets(lists):
    # Convert all lists to sets
    sets = [set(lst) for lst in lists]
    
    # Check subsets
    for i in range(len(sets) - 1):
        if not sets[i + 1].issubset(sets[i]):
            return False
    return True

# Check and print the result
result = check_subsets(all_full_assortment_index)
print("All subsequent lists are subsets of the previous one:", result)

All subsequent lists are subsets of the previous one: True


In [12]:
def filter_probability_frequency2(whole_choice_collection,purchase_samples,assortment_index):
    
    choice_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    frequency_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    for i in range(len(assortment_index)):
        choice_collection[:,i] = whole_choice_collection[:,assortment_index[i]]
        frequency_collection[:,i] = purchase_samples[assortment_index[i]]
        
    return choice_collection,frequency_collection

In [13]:
# generate instances that satisfy for assortment offertimes is >=20
## generate 50 random full instances 
# the observations with at least 20 times are >= 1e-3
pred_full_instance = []
pred_full_samples = []
#collection_20,offertimes_20,assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[0])

for j in range(pred_instance_size[0]):
    print(f'generating {j} th instance')
    # full instance generation 
    curr_whole_instance,curr_purchase_samples = whole_instance_generation(n,jd_collection,jd_offertimes)
    # filter choice probability and purchase frequency of each produt in each assortment
    curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])

    pred_full_instance.append(curr_whole_instance)
    pred_full_samples.append(curr_purchase_samples)
    
    ''' condition = False
    for x in range(len(all_full_collections[0])):
        for y in all_full_collections[0][x]:
            if curr_choice_collection[y][x]<1e-11:
                condition = True
    while condition:
        curr_whole_instance,curr_purchase_samples = whole_instance_generation(jd_collection,jd_offertimes)
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])

        condition = False
        for x in range(len(all_full_collections[0])):
            for y in all_full_collections[0][x]:
                if curr_choice_collection[y][x] < 1e-11:
                    condition = True
    if condition == True:
        print('instance_generation error')
    else:

            
        pred_full_instance.append(curr_whole_instance)
        pred_full_samples.append(curr_purchase_samples) '''

generating 0 th instance
generating 1 th instance
generating 2 th instance
generating 3 th instance
generating 4 th instance
generating 5 th instance
generating 6 th instance
generating 7 th instance
generating 8 th instance
generating 9 th instance
generating 10 th instance
generating 11 th instance
generating 12 th instance
generating 13 th instance
generating 14 th instance
generating 15 th instance
generating 16 th instance
generating 17 th instance
generating 18 th instance
generating 19 th instance
generating 20 th instance
generating 21 th instance
generating 22 th instance
generating 23 th instance
generating 24 th instance
generating 25 th instance
generating 26 th instance
generating 27 th instance
generating 28 th instance
generating 29 th instance
generating 30 th instance
generating 31 th instance
generating 32 th instance
generating 33 th instance
generating 34 th instance
generating 35 th instance
generating 36 th instance
generating 37 th instance
generating 38 th insta

In [14]:
def compute_lb_ub_w_ci(frequency_collection,choice_collection,target_z_score):
    
    stardard_error = np.zeros(frequency_collection.shape)
    for i in range(stardard_error.shape[0]):
        for j in range(stardard_error.shape[1]):
            if frequency_collection[i][j]!=0:
                stardard_error[i][j] = np.sqrt((1-choice_collection[i][j])/frequency_collection[i][j])
                
    lb = np.zeros(frequency_collection.shape)
    ub = np.zeros(frequency_collection.shape)
    for i in range(lb.shape[0]):
        for j in range(lb.shape[1]):
            if frequency_collection[i][j]!=0:
                lb[i][j] = choice_collection[i][j] * (1-target_z_score*stardard_error[i][j]) 
                ub[i][j] = choice_collection[i][j] * (1+target_z_score*stardard_error[i][j])
    
    return lb,ub

In [15]:
# full instance generation 
all_full_instances = []
all_full_lb = []
all_full_ub = []

# pre-determined confidence interval 
confidence_level = 0.995
alpha = 1 - confidence_level
# Find z-score for the given confidence level
target_z_score = stats.norm.ppf(1 - alpha / 2)  

for i in range(len(pred_offer_times_list)):
    
    full_instances = []
    full_lbs = []
    full_ubs = []
    print(f'check assortment index for offertimes {pred_offer_times_list[i]}')
    for j in range(pred_instance_size[i]):
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(pred_full_instance[j],pred_full_samples[j],all_full_assortment_index[i])
        # re-check if all choice probability are >= 1e-3
        ''' condition = False
        for x in range(len(all_full_collections[i])):
            for y in all_full_collections[i][x]:
                if curr_choice_collection[y][x] < 1e-3:
                    condition = True
        if condition == True:
            print('instance_generation error') '''
        
        # compute the collection of lower bound and upper bound l_ij and u_ij
        curr_lb,curr_ub = compute_lb_ub_w_ci(curr_frequency,curr_choice_collection,target_z_score)
        
        full_instances.append(curr_choice_collection)
        full_lbs.append(curr_lb)
        full_ubs.append(curr_ub)
    
    all_full_instances.append(full_instances)
    all_full_lb.append(full_lbs)
    all_full_ub.append(full_ubs)
    



check assortment index for offertimes 20
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation error
instance_generation err

In [16]:
# train-test split
all_train_instances = []
all_train_collection = []
all_test_instance = []
all_test_collection = []
all_train_lb = []
all_train_ub = []
all_train_offertimes = []

for i in range(len(pred_instance_size)):
    train_instances_collection = []
    train_collection = []
    train_lb_collection = []
    train_ub_collection = []
    train_offertimes_collection = []
    
    test_instance_collection = []
    test_collection = []
    
    test_index_collection = []
    
    
    for j in range(pred_instance_size[i]):
        
        curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
        curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
        curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
        
        curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
        
        curr_train_collection = []
        curr_test_collection = []
        curr_offertimes_lst = []
        
        while(len(curr_train_collection)==0):
            
            chosen_test_idx = random.sample(range(len(all_full_collections[i])),pred_test_collection_size[i])
            chosen_test_idx.sort()
            
            chosen_train_idx = [x for x in range(len(all_full_collections[i])) if x not in chosen_test_idx ]
    
            for k in range(len(chosen_train_idx)):
                curr_train_collection.append(all_full_collections[i][chosen_train_idx[k]])
                curr_train_instance[:,k] = all_full_instances[i][j][:,chosen_train_idx[k]]
                curr_train_lb[:,k] = all_full_lb[i][j][:,chosen_train_idx[k]]
                curr_train_ub[:,k] = all_full_ub[i][j][:,chosen_train_idx[k]]
                
                curr_offertimes_lst.append(all_full_offertimes[i][chosen_train_idx[k]])
                
            for k in range(len(chosen_test_idx)):
                curr_test_collection.append(all_full_collections[i][chosen_test_idx[k]])
                curr_test_instance[:,k] = all_full_instances[i][j][:,chosen_test_idx[k]]
                
            ## check if the training instance includes all the testing products in the test instance
            # ensure each test product has been offered at least  once
            new_lst_train = sum(curr_train_collection,[])
            unique_numbers_train = list(set(new_lst_train)) 
            
            new_lst_test = sum(curr_test_collection,[])
            unique_numbers_test = list(set(new_lst_test)) 
            
            for num in unique_numbers_test:
                if num not in unique_numbers_train:
                    #print('the unique train numbers',unique_numbers_train)
                    #print('the unique test numbers',unique_numbers_test)
                    curr_train_collection = []
                    curr_test_collection = []
                    curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                    curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                    curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                    curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                    curr_offertimes_lst = []
            if chosen_test_idx in test_index_collection:
                #print('check if exisits repeated indexes ')
                curr_train_collection = []
                curr_test_collection = []
                curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                curr_offertimes_lst = []
            elif len(curr_test_collection)>0:
                test_index_collection.append(chosen_test_idx)
        #print('chosen training indexes',chosen_train_idx)
        #print('chosen testing indexes',chosen_test_idx)
        #print('curr train collection\n',curr_train_collection)
        #print('curr test collection\n',curr_test_collection)
        ''' df_train = pd.DataFrame(curr_train_instance)
        df_test = pd.DataFrame(curr_test_instance)
        df_offertimes = pd.DataFrame(curr_offertimes_lst) '''
        #print('curr train instance\n',df_train)
        #print('curr test instance\n',df_test)
        
        ## output the training and testing instance
        ''' df_train.to_csv('instances/train_instances/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_test.to_csv('instances/test_instances/test_'+str(pred_offer_times_list[i])+'/test_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_offertimes.to_csv('instances/train_offertimes/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv') '''
        
        train_instances_collection.append(curr_train_instance)
        train_collection.append(curr_train_collection)
        train_lb_collection.append(curr_train_lb)
        train_ub_collection.append(curr_train_ub)
        train_offertimes_collection.append(curr_offertimes_lst)
                
        test_instance_collection.append(curr_test_instance)
        test_collection.append(curr_test_collection)
        
    all_train_instances.append(train_instances_collection)
    all_train_collection.append(train_collection)
    all_train_lb.append(train_lb_collection)
    all_train_ub.append(train_ub_collection)
    all_train_offertimes.append(train_offertimes_collection)
    all_test_instance.append(test_instance_collection)
    all_test_collection.append(test_collection)    
        

In [17]:
all_test_collection[0][0]

[[0, 5], [0, 3, 7], [0, 5, 6], [0, 5, 8], [0, 6, 8]]

In [25]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

def mem_mle_pyo(data, collection,offertimes):
    n,m = data.shape
    model = pyo.ConcreteModel()
    model.v = pyo.Var(range(n), bounds=(-100,100))
    model.alpha = pyo.Var(range(n),domain=pyo.NonNegativeReals, bounds=(0.01,10))
    model.lam = pyo.Var(range(m), bounds=(0,50))
    
    def obj_rule(model):
        obj = 0
        for i in range(len(collection)):
            for j in collection[i]:
                obj += offertimes[i] * data[j][i] * model.alpha[j] * (model.v[j] - model.lam[i])
        return obj
    
    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)
    
    model.assortment_constraints = pyo.ConstraintList()
    for i in range(m):
        model.assortment_constraints.add(sum(pyo.exp(model.alpha[j] * (model.v[j] - model.lam[i]) ) for j in collection[i])==1)
    
    # to scale the preference of the outside option to be 1 
    # to avoid overfitting   
    model.alpha_constraint = pyo.Constraint(expr=model.alpha[0] == 1)
    model.v_constraint = pyo.Constraint(expr=model.v[0] == 0)
    
    opt = pyo.SolverFactory('ipopt')  # use iterior point method to solve continuous optimizatio problem 
    results = opt.solve(model)
    #results = opt.solve(model)
    #model.pprint()
    if results.solver.status == pyo.SolverStatus.ok:
    #if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        v_values = [model.v[i].value for i in range(n)]
        alpha_values = [model.alpha[i].value for i in range(n)]
        lam_values = [model.lam[i].value for i in range(m)]
        prob = np.zeros((n,m)) 
        for j in range(m):
            for i in collection[j]:
                prob[i,j] = np.exp(alpha_values[i]*(v_values[i]-lam_values[j]))       
        obj_value = model.obj()
        
        return v_values,alpha_values,lam_values,prob, obj_value,results.solver.time
    else:
        raise ValueError("MEM MLE: Solver failed to find an optimal solution.")

# Example usage:
# v_values, obj_value = mem_mle(data, collection)


In [19]:
def binarySearch(lb,ub,v,assortment,x,alpha):
    
    mid = (lb+ub)/2
    for j in assortment:
        x[j] = 1-expon.cdf(mid-v[j],scale=1/alpha[j]) # pdf = lambda * exp(-lambda * x). This parameterization corresponds to using scale = 1 / lambda.
    
    if np.abs(sum(x)-1)<0.000001:
        return x
    elif sum(x)>1:
        return binarySearch(mid,ub,v,assortment,x,alpha)
    else:
        return binarySearch(lb,mid,v,assortment,x,alpha)

In [20]:
def collection_distribution_mem(n,collection,v,alpha):
    
    collection_distribution = np.zeros((n,len(collection)))

    lb = -2000
    ub = 2000
    for i in range(len(collection)):
        prob = np.zeros(n)
        collection_distribution[:,i] = binarySearch(lb,ub,v,collection[i],prob,alpha)
        #collection_distribution[:,i] = bisection(prob,n,collection[i],v,alpha,lb,ub)

    return collection_distribution

In [21]:
## prediction for the current testing collection of assortments 
def mem_prediction(curr_test_instance,curr_test_collection,price,v_values,alpha_values):
    ''' true assortment ranking '''
    true_choice_prob = [] 
    true_revenue = []
    
    mem_revenue = []
    mem_sales = []
    
    mem_runtime = []
    
    
    # mle estimate
    start_time = time.time()
    estimated_oos_prob = collection_distribution_mem(len(v_values),curr_test_collection,v_values,alpha_values)
    used_t_1 = time.time() - start_time
    
    for k in range(len(curr_test_collection)):

        # true revenue and true choice probabilities for current testing assortment
        curr_revenue = np.dot(curr_test_instance[:,k],price)
        true_revenue.append(curr_revenue)
        true_choice_prob.append(curr_test_instance[:,k])
        
        st_time_2 = time.time()
        curr_mem_rev = np.dot(estimated_oos_prob[:,k],price)
        used_t_2 = time.time() - st_time_2
        
        mem_sales.append(estimated_oos_prob[:,k])
        mem_revenue.append(curr_mem_rev)
        print(f'revenue of assortment {curr_test_collection[k]} is {curr_mem_rev}')
        
        ## record the runtime result
        mem_runtime.append(used_t_1+used_t_2)
        
    return true_revenue,true_choice_prob,mem_revenue,mem_sales,mem_runtime

In [22]:
def kendall_tau_distance(values1, values2):
    """Compute the Kendall tau distance."""
    num = len(values1)
    assert len(values2) == num, "Both lists have to be of equal length"
    i, j = np.meshgrid(np.arange(num), np.arange(num))
    a = np.argsort(values1)
    b = np.argsort(values2)
    ndisordered = np.logical_or(np.logical_and(a[i] < a[j], b[i] > b[j]), np.logical_and(a[i] > a[j], b[i] < b[j])).sum()
    return ndisordered/2 

In [28]:
## record representability of all instances

## record the kendall tau distance between the predicted ranking of mem and the truth
all_mem_distance_list = []  

## record the true revenue of the predicted best assortment
all_best_true_rev_list = []
all_best_rev_mem_list = []  

## record the difference in best assortment
all_mem_best_rev_diff = []

## record all the true and predicted revenues
all_true_revenue_list = []
all_mem_revenue_list = []

all_true_prob_list = [] 
all_mem_sales_list = []

all_true_ranking = []
all_mem_ranking = []

## record the runtime of mem prediction
all_mem_pred_runtime = []

for i in range(len(all_full_collections)):
       
    # define as a container for quantity of the same collection size 
    # The following lists are of the same size as the instance size
    
    true_best_rev_collection = [] # a container for groundtruth best reveue 
    mem_best_rev_collection = [] # a container for the best revenue predicted by robust mem
    
    true_ranking_collection = [] # a container for groundtruth reveue ranking 
    mem_ranking_collection = [] # a container for ranking by robust mem
    
    mem_distance_collection = [] # a container for kendall tau distance between true ranking and the ranking by robust mem
   
    
    true_choice_prob_collection = []
    mem_sales_frac_collection = []

    true_revenue_collection = []
    mem_revenue_collection = []
    
    mem_runtime_collection = []
    
    for j in range(pred_instance_size[i]):
        
        ## mle 
        print(f'testing for offertimes {pred_offer_times_list[i]}, {j}th instance')
        # v_values,alpha_values,lam_values,prob, obj_value,results.solver.time
        v_values,alpha_values,lambda_vaues,prob,ll_value,ll_time = mem_mle_pyo(all_train_instances[i][j],all_train_collection[i][j],all_train_offertimes[i][j])
        ########### mem prediction#################
        # mem_prediction(curr_test_instance,curr_test_collection,price,v_values)
        true_revenue,true_choice_prob,mem_revenue,mem_sales,mem_runtime = mem_prediction(all_test_instance[i][j],all_test_collection[i][j],price,v_values,alpha_values)
        # the following lists are of the same size as number of testing assortments
        
        for x in range(len(mem_runtime)):
            mem_runtime[x] = mem_runtime[x] + ll_time
        
        true_revenue_collection.append(true_revenue)
        true_choice_prob_collection.append(true_choice_prob)
        
        mem_revenue_collection.append(mem_revenue)   
        mem_sales_frac_collection.append(mem_sales)
        mem_runtime_collection.append(mem_runtime)
         
        # true assortment ranking for test instance [i][j]
        curr_true_rank = np.argsort(-np.array(true_revenue))  
        true_ranking_collection.append(curr_true_rank)
        # true best assortment revenue for test instance [i][j]
        true_best_rev_collection.append(true_revenue[curr_true_rank[0]])
        
        # lb mdm assortment ranking for test instance [i][j] 
        curr_mem_rank  = np.argsort(-np.array(mem_revenue)) 
        mem_ranking_collection.append(curr_mem_rank)
        mem_best_rev_collection.append(true_revenue[curr_mem_rank[0]])
        
        
        #  mem kendallTau Distance for current instance [i][j]
        curr_mem_dist = kendall_tau_distance(curr_true_rank, curr_mem_rank)
        mem_distance_collection.append(curr_mem_dist)
    
    
    # for each test instance [i][j] kendall tau distance is a number
    all_mem_distance_list.append(mem_distance_collection)
    
    # for each test instance [i][j] best revenue is a number
    all_best_true_rev_list.append(true_best_rev_collection) 
    all_best_rev_mem_list.append(mem_best_rev_collection)
    
    # for each test instance [i][j] best revenue difference is a number
    all_mem_best_rev_diff.append((np.array(true_best_rev_collection) - np.array(mem_best_rev_collection))/ np.array(true_best_rev_collection))
    
    # for each test instance [i][j], true revenue is a list of the same size as the number of testing assortments
    all_true_revenue_list.append(true_revenue_collection)
    
    all_mem_revenue_list.append(mem_revenue_collection)
    
    all_true_prob_list.append(true_choice_prob_collection)
    all_mem_sales_list.append(mem_sales_frac_collection)
 
    # for each test instance [i][j], true ranking is a list of the same size as the number of testing assortments
    all_true_ranking.append(true_ranking_collection)
    all_mem_ranking.append(mem_ranking_collection)

    ## for each test instance [i][j], runtime is a list of the same size as the number of testing assortments
    all_mem_pred_runtime.append(mem_runtime_collection)

testing for offertimes 20, 0th instance


revenue of assortment [0, 5] is 0.33261989818725207
revenue of assortment [0, 3, 7] is 0.1409799156573528
revenue of assortment [0, 5, 6] is 0.2843719038056279
revenue of assortment [0, 5, 8] is 0.3899440578089485
revenue of assortment [0, 6, 8] is 0.2675189874805435
testing for offertimes 20, 1th instance
revenue of assortment [0, 3] is 0.14019371379808085
revenue of assortment [0, 4] is 0.36327063227591044
revenue of assortment [0, 1, 4] is 0.36308780590589085
revenue of assortment [0, 3, 4] is 0.17789204790260976
revenue of assortment [0, 5, 6] is 0.2711038725169845
testing for offertimes 20, 2th instance
revenue of assortment [0, 1, 8] is 0.5189857998209946
revenue of assortment [0, 2, 6] is 0.1642487619091868
revenue of assortment [0, 3, 4] is 0.3908081936002093
revenue of assortment [0, 4, 5] is 0.5197695136944318
revenue of assortment [0, 5, 8] is 0.5313726857989772
testing for offertimes 20, 3th instance
revenue of assortment [0, 1] is 0.2589638998649766
revenue of assortment [

In [29]:
## adding details 1
for i in range(len(pred_offer_times_list)):
    df_mem_details1 = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mem_dist':all_mem_distance_list[i],
                                    'true_best_rev':all_best_true_rev_list[i],'mem_best_rev':all_best_rev_mem_list[i],
                                    'mem_best_rev_diff':all_mem_best_rev_diff[i]})
    df_mem_details1.to_csv('prediction/mem/revenue_prediction/details1/'+str(pred_offer_times_list[i])+'.csv')

In [30]:
## adding details 2 about ranking, revenue, choice probability
for i in range(len(pred_offer_times_list)):
    for j in range(pred_instance_size[i]):
        
        true_rank = all_true_ranking[i][j]
        mem_rank = all_mem_ranking[i][j]
        df_rank = pd.DataFrame({'true_rank':true_rank,'mem_rank':mem_rank})
        df_rank.to_csv('prediction/mem/revenue_prediction/details2/ranking/offertimes'+str(pred_offer_times_list[i])+'/rank_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        true_rev = all_true_revenue_list[i][j]
        mem_rev = all_mem_revenue_list[i][j]
        df_rev = pd.DataFrame({'true_rev':true_rev,'mem_rev':mem_rev})
        df_rev.to_csv('prediction/mem/revenue_prediction/details2/revenue/offertimes'+str(pred_offer_times_list[i])+'/rev_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
    
        mem_runtime = all_mem_pred_runtime[i][j]
        df_runtime = pd.DataFrame({'mem_runtime':mem_runtime})
        df_runtime.to_csv('prediction/mem/revenue_prediction/details2/runtime/offertimes'+str(pred_offer_times_list[i])+'/runtime_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        
        ## using sales fraction prediction
        ## record the choice probability prediction interval
        ''' true_prob = all_true_prob_list[i][j]
        df_true_prob = pd.DataFrame(true_prob)
        
        mem_sales = all_mem_sales_list[i][j]
        df_mem_sales = pd.DataFrame(mem_sales).T
 
        df_true_prob.to_csv('prediction/mem/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/true/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_mem_sales.to_csv('prediction/mem/prob_prediction/offertimes'+str(pred_offer_times_list[i])+'/mem/prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv') '''

In [31]:
## avg prediction mem runtime 
all_mem_pred_avg_runtime = []

for i in range(len(pred_offer_times_list)):
    avg_mem_pred_runtime_collection = []
    
    for j in range(pred_instance_size[i]): 
        avg_mem_pred_runtime_collection.append(np.mean(np.array(all_mem_pred_runtime[i][j]))) 
        
    all_mem_pred_avg_runtime.append(np.mean(np.array(avg_mem_pred_runtime_collection)))

avg_mem_dist = []
avg_mem_best_rev_diff = []

for i in range(len(pred_offer_times_list)):
    
    avg_mem_dist.append(np.mean(all_mem_distance_list[i]))
    avg_mem_best_rev_diff.append(np.mean(all_mem_best_rev_diff[i]))
    

df_mem_jd_summary = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'avg_mem_dist':avg_mem_dist,'avg_mem_best_rev_diff':avg_mem_best_rev_diff,
                                   'avg_mem_pred_runtime':all_mem_pred_avg_runtime})
df_mem_jd_summary.to_csv('prediction/mem/jd_mem_summary.csv')

In [32]:
df_mem_jd_summary

,offertimes,avg_mem_dist,avg_mem_best_rev_diff,avg_mem_pred_runtime
0,20,1.72,0.073306,0.448551
1,30,0.88,0.047178,0.460338
2,40,0.34,0.037203,0.434622
3,50,0.26,0.008152,0.445485
4,60,0.06,0.024551,0.431409


In [33]:
def mem_limit(data,collection,offertimes):
    #v,ll = mem_mle(curr_instance, collection, binary_collection)
    start_time = time.time()
    v_values,alpha_values,lambda_vaues,mem_limit_prob,ll_value,ll_time = mem_mle_pyo(data,collection,offertimes)
    #return v_values,alpha_values,lam_values,prob, obj_value,results.solver.time

    mem_limit_loss = 0
    for i in range(len(collection)):
        for j in range(n):
            weight = data[j][i] * offertimes[i]
            mem_limit_loss = mem_limit_loss + weight*np.abs(data[j][i] - mem_limit_prob[j][i])       
    runtime = time.time() - start_time 
    
    return mem_limit_loss, mem_limit_prob, ll_time

In [34]:

## record the limit loss of all instances
all_mem_limit_loss = []
all_mem_limit_prob = []
all_mem_limit_runtime = []

for i in range(len(pred_offer_times_list)):

    
    ## define container for mem limit 
    mem_limit_loss_collection =[]
    mem_limit_runtime_collection =[]
    mem_limit_probability_collection =[]
    
    for j in range(pred_instance_size[i]):
        

        mem_limit_loss, mem_limit_prob,mem_limit_runtime = mem_limit(all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i])  
        mem_limit_probability_collection.append(mem_limit_prob)
        mem_limit_loss_collection.append(mem_limit_loss)
        mem_limit_runtime_collection.append(mem_limit_runtime)

    
    ## for each train instance [i][j] limit loss result is a number
    all_mem_limit_loss.append(mem_limit_loss_collection)
    all_mem_limit_runtime.append(mem_limit_runtime_collection)
    
    ## for each train instance [i][j] limit probability is a matrix
    all_mem_limit_prob.append(mem_limit_probability_collection)

In [36]:
## adding limit results
ins_idx = list(range(pred_instance_size[0]))
for i in range(len(pred_offer_times_list)):
    df_mem_loss = pd.DataFrame({'ins_idx':ins_idx,'mem_loss':all_mem_limit_loss[i],'avg_loss':all_mem_limit_loss[i]/sum(all_full_offertimes[i]),  'mem_limit_time':all_mem_limit_runtime[i]})
    df_mem_loss.to_csv('limit/mem/limit/'+str(pred_offer_times_list[i])+'.csv')
    
    for j in range(pred_instance_size[i]):
        df_limit_prob = pd.DataFrame(all_mem_limit_prob[i][j])
        df_limit_prob.to_csv('limit/mem/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')

In [41]:
all_avg_loss = []
for i in range(len(pred_offer_times_list)):
    all_avg_loss.append(all_mem_limit_loss[i]/sum(all_full_offertimes[i]))

In [42]:


avg_total_limit_loss = []
avg_toal_limit_loss_se = []
avg_mem_limit_runtime = []

#avg_loss = []
avg_loss_se = []

for i in range(len(pred_offer_times_list)):

    
    avg_total_limit_loss.append(np.mean(all_mem_limit_loss[i]))
    avg_mem_limit_runtime.append(np.sum(all_mem_limit_runtime[i]))
    avg_toal_limit_loss_se.append(np.std(all_mem_limit_loss[i])/np.sqrt(len(all_mem_limit_loss[i])))
    
    #avg_loss = np.mean(all_avg_loss[i])
    avg_loss_se.append(np.std(all_avg_loss[i])/np.sqrt(len(all_avg_loss[i])))

In [43]:
avg_loss = []
for i in range(len(avg_total_limit_loss)):
    avg_loss.append(avg_total_limit_loss[i]/sum(all_full_offertimes[i]))

In [44]:
df_mem_limit = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_mem_loss':avg_total_limit_loss,'total_mem_loss_se':avg_toal_limit_loss_se,
                                   'avg_mem_loss':avg_loss,'avg_mem_loss_se':avg_loss_se,
                                   'avg_mem_limit_runtime':avg_mem_limit_runtime
                                   })
df_mem_limit.to_csv('limit/mem/mem_limit_summary.csv')
df_mem_limit

,offertimes,total_mem_loss,total_mem_loss_se,avg_mem_loss,avg_mem_loss_se,avg_mem_limit_runtime
0,20,95.599340,2.800617,0.012488,0.000366,18.408008
1,30,81.672053,2.840935,0.010848,0.000377,15.822888
2,40,64.480302,2.683650,0.008760,0.000365,16.129484
3,50,48.582520,2.395156,0.006757,0.000333,16.318681
4,60,38.993648,2.312811,0.005506,0.000327,22.464432


In [45]:
for i in range(len(avg_total_limit_loss)):
    print(avg_total_limit_loss[i]/sum(all_full_offertimes[i]))

0.012488483347657607
0.010847662776891701
0.008759720467869799
0.006756956912043951
0.005506022051924749
